# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [30]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

In [31]:
# 2) seu código aqui
sinasc.isna().sum().sort_values(ascending=False)

In [32]:
# 3) seu código aqui
sinasc_df = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_df.isna().sum().sort_values(ascending=False)

In [33]:
# 4) seu código aqui
sinasc_df.shape

In [34]:
sinasc_df = sinasc_df.dropna(subset= ['APGAR5'])
sinasc_df.shape

In [35]:
# 5) seu código aqui
sinasc_df[['ESTCIVMAE', 'CONSULTAS']].head()

In [36]:
sinasc_df.loc[:, ['ESTCIVMAE', 'CONSULTAS']] = sinasc_df[['ESTCIVMAE', 'CONSULTAS']].fillna(9)
sinasc_df['ESTCIVMAE'].unique()

In [37]:
# 6) Seu código aqui
sinasc_df['QTDFILVIVO'].isna().sum()


In [38]:
sinasc_df.loc[:,'QTDFILVIVO'] = sinasc_df['QTDFILVIVO'].fillna(0)
sinasc_df['QTDFILVIVO'].isna().sum()

In [39]:
# 7) seu código aqui
sinasc_df[['LOCNASC', 'IDADEMAE', 'ESCMAE',
           'GESTACAO', 'GRAVIDEZ']].isna().sum()

Determinado que apenas as variáveis ESCMAE, GESTACAO e GRAVIDEZ, possuem valores nulos. GESTACAO e GRAVIDEZ parecem relevantes para determinar o APGAR, Como nossa base possue 27000 registros, dropar essas variáveis não será de grande impacto. Já ESCMAE não parece que terá grande relevância, portanto vale a pena preservar os registros que esse dado seja nulo, portanto substituiremos por 9: Ignorado, como é descrito na documentação.

In [40]:
sinasc_df.loc[:,'ESCMAE'] = sinasc_df['ESCMAE'].fillna(9)
sinasc_df = sinasc_df.dropna(subset=['GESTACAO', 'GRAVIDEZ'])
sinasc_df[['LOCNASC', 'IDADEMAE', 'ESCMAE',
           'GESTACAO', 'GRAVIDEZ']].isna().sum()

In [43]:
# 8) seu código aqui
sinasc_df.loc[sinasc_df['APGAR5'] <=10,'APGAR5_CAT'] = 'normal'
sinasc_df.loc[sinasc_df['APGAR5'] <=7,'APGAR5_CAT'] = 'asfixia leve'
sinasc_df.loc[sinasc_df['APGAR5'] <=5,'APGAR5_CAT'] = 'asfixia moderada'
sinasc_df.loc[sinasc_df['APGAR5'] <=3,'APGAR5_CAT'] = 'asfixia severa'
sinasc_df['APGAR5_CAT'].value_counts()

In [45]:
# 9) seu código aqui
novos_nomes = {
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar_5',
    'APGAR5_CAT': 'apgar_5_cat'
}

# Renomeia as colunas
sinasc_df.rename(columns=novos_nomes, inplace=True)

# Verifica o resultado
print(sinasc_df.columns)
sinasc_df
